# Mem0 Integration with Codebase Agent

This notebook demonstrates how to implement mem0 (long-term memory) with Google Gemini in your Codebase Agent project. Mem0 allows your AI assistant to remember previous conversations and provide context-aware responses.

## Key Features
- **Persistent Memory**: Conversations are stored and can be retrieved across sessions
- **Semantic Search**: Find relevant memories based on context, not just keywords
- **Enhanced Context**: AI responses become more personalized and contextually aware
- **Repository-Specific Memory**: Different memory contexts for different code repositories

Let's explore how to integrate this powerful memory system into your existing codebase analysis tool!

## 1. Setup: Install Required Packages

First, we need to install the required packages for Google Gemini and mem0. Make sure you have a Google API key set in your environment variables as `GEMINI_API_KEY`.

In [ ]:
# Install required packages
!pip install google-genai mem0ai --upgrade

# Also install any missing dependencies
!pip install python-dotenv

## 2. Initialize Google Gemini and Mem0 Memory

Now we'll set up the Google Gemini client and configure mem0 with Gemini as both the LLM and embedding model. We'll use a local Qdrant instance as the vector store.

In [ ]:
import os
from google import genai
from mem0 import Memory
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Create Gemini client
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

# Configure mem0 with Gemini models
config = {
    "embedder": {
        "provider": "gemini",
        "config": {
            "model": "models/text-embedding-004",
        }
    },
    "llm": {
        "provider": "gemini",
        "config": {
            "model": "gemini-2.5-flash",
            "temperature": 0.0,
            "max_tokens": 2000,
        }
    },
    "vector_store": {
        "config": {
            "embedding_model_dims": 768,
        }
    }
}

# Initialize memory system
memory = Memory.from_config(config)
print("✅ Memory system initialized successfully!")

## 3. Store User Memories from Example Conversations

Let's simulate a developer working with our codebase agent by adding some example conversations to memory. These will demonstrate how the system remembers user preferences and project context.

In [ ]:
# Example conversation 1: Repository analysis preferences
conv1 = [
    {"role": "user", "content": "I just uploaded my FastAPI project to analyze. I'm particularly interested in understanding the database architecture and API endpoint organization. I prefer detailed Mermaid diagrams that show the data flow."},
    {"role": "assistant", "content": "I've analyzed your FastAPI project. The database architecture uses SQLAlchemy with a clear separation between models, schemas, and database operations. Your API endpoints are well-organized with proper dependency injection. I've generated a detailed Mermaid diagram showing the data flow from HTTP requests through your router layers to the database. The diagram highlights how your authentication middleware intercepts requests and how your CRUD operations are structured."},
    {"role": "user", "content": "Perfect! Can you also identify potential areas where I could add caching? I'm using Redis for session storage but wondering about query caching."},
    {"role": "assistant", "content": "Based on your codebase structure, I see several opportunities for query caching. Your user lookup operations in the authentication service would benefit from Redis caching, especially the `get_user_by_email` function. I also notice your analytics endpoints perform complex aggregations that could be cached. I recommend adding a caching layer in your service classes, particularly for read-heavy operations like dashboard data retrieval."}
]

# Add to memory for user "developer_sarah"
result1 = memory.add(conv1, user_id="developer_sarah")
print("✅ Added conversation 1 about FastAPI project analysis")

# Example conversation 2: Feature implementation preferences  
conv2 = [
    {"role": "user", "content": "I want to add a real-time notification system to my project. I prefer using WebSockets over Server-Sent Events, and I like to keep all real-time logic in separate modules for better organization."},
    {"role": "assistant", "content": "Excellent choice! For your FastAPI project, I recommend implementing WebSockets using FastAPI's built-in support. Based on your preference for modular organization, I suggest creating a `realtime` package with separate modules: `websocket_manager.py` for connection handling, `notification_service.py` for business logic, and `events.py` for event definitions. This aligns with your existing project structure where you separate concerns clearly."},
    {"role": "user", "content": "That sounds perfect. I also want to make sure the notifications are persisted to the database even if users are offline. Should I use a message queue?"},
    {"role": "assistant", "content": "Given your SQLAlchemy setup, I recommend adding a `notifications` table to store persistent notifications. For the message queue, Redis (which you're already using) with rq or Celery would be ideal. This way you can queue notification tasks, store them in the database, and deliver them via WebSocket when users come online. The architecture would fit well with your existing database patterns."}
]

result2 = memory.add(conv2, user_id="developer_sarah")
print("✅ Added conversation 2 about WebSocket implementation")

# Example conversation 3: Code quality preferences
conv3 = [
    {"role": "user", "content": "I'm working on improving code quality in my repositories. I really value comprehensive type hints, detailed docstrings, and consistent error handling patterns. Can you analyze my latest commit and suggest improvements?"},
    {"role": "assistant", "content": "I've analyzed your recent commits and I'm impressed with your typing discipline! Your error handling is quite good, though I notice some inconsistency in exception types. I recommend creating custom exception classes that inherit from HTTPException for API-specific errors. For docstrings, you're doing well with most functions, but some utility functions could use more detailed parameter descriptions. Your type hints are excellent - you're already using Union types and Optional properly."},
    {"role": "user", "content": "Thanks! I do tend to rush the utility functions. What about testing? I'm trying to get better at writing comprehensive tests."},
    {"role": "assistant", "content": "Your test structure is solid! I see you're using pytest with good fixture organization. One suggestion: your API tests could benefit from more edge case coverage, particularly around authentication and input validation. Consider adding tests for malformed requests and expired tokens. Also, your database tests are thorough, but adding some property-based tests using Hypothesis could help catch edge cases in your data validation logic."}
]

result3 = memory.add(conv3, user_id="developer_sarah")  
print("✅ Added conversation 3 about code quality preferences")

print(f"\n🎉 Successfully stored {len([conv1, conv2, conv3])} conversations in memory!")

## 4. Search and Retrieve Memories

Now let's see how we can search through the stored memories to find relevant information about the user's preferences and past interactions.

In [ ]:
# Search for information about the user's preferences
print("🔍 Searching for user preferences...")
user_preferences = memory.search(query="What are the user's preferences?", user_id="developer_sarah")

print("\n📋 User Preferences Found:")
for i, memory_item in enumerate(user_preferences["results"][:5], 1):
    print(f"{i}. {memory_item['memory']}")

print("\n" + "="*60)

# Search for technical preferences
print("\n🔍 Searching for technical implementation preferences...")
tech_preferences = memory.search(query="technical preferences implementation patterns", user_id="developer_sarah")

print("\n🛠️ Technical Preferences:")
for i, memory_item in enumerate(tech_preferences["results"][:3], 1):
    print(f"{i}. {memory_item['memory']}")

print("\n" + "="*60)

# Search for past project analysis
print("\n🔍 Searching for past project analysis...")
project_info = memory.search(query="FastAPI project database", user_id="developer_sarah")

print("\n📊 Project Analysis History:")
for i, memory_item in enumerate(project_info["results"][:3], 1):
    print(f"{i}. {memory_item['memory']}")

print(f"\n📈 Total memories found: {len(user_preferences['results'])}")

## 5. Compare Gemini Responses With and Without Memory

Let's see the difference between Gemini's responses when it has no context versus when it has access to the user's memory and preferences.

In [ ]:
# Test prompt about implementing a new feature
test_prompt = "I want to add logging to my application. Can you suggest the best approach and implementation strategy?"

print("🤖 Testing Gemini Response WITHOUT Memory Context")
print("="*60)

# Response without memory
basic_system_prompt = "You are a helpful AI assistant that helps with code architecture and implementation."

response_no_memory = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=test_prompt,
    config={
        "system_instruction": basic_system_prompt
    }
)

print("Response without memory:")
print(response_no_memory.text)

print("\n" + "="*60)
print("🧠 Testing Gemini Response WITH Memory Context")
print("="*60)

# Retrieve relevant memories for the prompt
relevant_memories = memory.search(query=test_prompt, user_id="developer_sarah")
memories_str = "\n".join(f"- {entry['memory']}" for entry in relevant_memories["results"])

# Enhanced system prompt with memory
enhanced_system_prompt = f"""You are a helpful AI assistant that helps with code architecture and implementation.

Based on previous conversations, here's what you know about the user:
{memories_str}

Use this context to provide personalized recommendations that align with their preferences and project structure."""

response_with_memory = client.models.generate_content(
    model="gemini-2.5-flash", 
    contents=test_prompt,
    config={
        "system_instruction": enhanced_system_prompt
    }
)

print("Response with memory context:")
print(response_with_memory.text)

print("\n" + "="*60)
print("📊 COMPARISON SUMMARY:")
print("Without memory: Generic logging advice")
print("With memory: Personalized recommendations based on user's FastAPI project, modular preferences, and code quality standards")

## 6. Interactive Long-term Memory Chatbot

Now let's create an interactive chatbot that maintains memory across conversations. This simulates how your codebase agent can remember user preferences and project context across multiple sessions.

In [ ]:
def chat_with_memory_codebase_agent(history: list, user_id: str = "developer_sarah") -> list:
    """Enhanced chatbot with memory for codebase analysis"""
    
    # Get the latest user message
    latest_message = history[-1]["parts"][0]["text"]
    print(f"🔍 User: {latest_message}")
    
    # Retrieve relevant memories
    relevant_memories = memory.search(query=latest_message, user_id=user_id, limit=5)
    memories_str = "\n".join(f"- {entry['memory']}" for entry in relevant_memories["results"])
    
    # Enhanced system prompt for codebase agent
    memory_system_prompt = f"""You are an expert code architecture assistant with access to the user's conversation history and preferences.

User Context and Preferences:
{memories_str}

Guidelines:
- Use the user's past preferences to inform your recommendations
- Reference their previous projects and patterns when relevant
- Maintain consistency with their coding standards and architectural choices
- Provide specific, actionable advice based on their tech stack and preferences
- Remember their project structure and suggest improvements that align with their existing patterns

Answer based on the conversation history and user preferences."""
    
    # Generate response with memory context
    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=history,
        config={"system_instruction": memory_system_prompt}
    )
    
    # Add assistant response to history
    history.append({"role": "model", "parts": [{"text": response.text}]})
    
    # Convert history to memory format and store
    messages = []
    for i, turn in enumerate(history):
        role = "user" if turn["role"] == "user" else "assistant"
        content = turn["parts"][0]["text"]
        messages.append({"role": role, "content": content})
    
    # Add new conversation to memory
    memory.add(messages, user_id=user_id)
    
    print(f"🤖 Assistant: {response.text}")
    return history

def run_codebase_agent_chat():
    """Interactive chat session with memory"""
    print("🚀 Codebase Agent with Memory - Type 'exit' to quit")
    print("💡 Try asking about: code architecture, implementing features, or improving code quality")
    print("-" * 60)
    
    history = []
    
    while True:
        user_input = input("\n💭 You: ").strip()
        
        if user_input.lower() == 'exit':
            print("👋 Goodbye! Your conversation has been saved to memory.")
            break
            
        # Add user message to history
        history.append({"role": "user", "parts": [{"text": user_input}]})
        
        # Get response with memory
        history = chat_with_memory_codebase_agent(history)
        print("-" * 60)

# Run the interactive chat
# Uncomment the line below to start the interactive session
# run_codebase_agent_chat()

print("🎯 Interactive chatbot function created!")
print("📝 To start chatting, uncomment and run: run_codebase_agent_chat()")
print("🧪 Or test with a single message:")

# Test with a single message
test_history = [{"role": "user", "parts": [{"text": "I'm starting a new microservices project. Based on our previous discussions, what architecture would you recommend?"}]}]
final_history = chat_with_memory_codebase_agent(test_history)

print("\n✅ Demo completed! The assistant remembered the user's preferences for FastAPI, modular organization, and code quality standards.")

## 🎉 Conclusion

Congratulations! You've successfully integrated mem0 (long-term memory) with Google Gemini into your codebase agent. Here's what we accomplished:

1. **Memory Storage**: Stored user conversations and preferences 
2. **Context Retrieval**: Retrieved relevant memories for enhanced responses
3. **Semantic Search**: Retrieved relevant memories based on context and meaning
4. **Enhanced Responses**: Generated personalized responses using memory context
5. **Interactive Chat**: Created a chatbot that remembers user preferences and project history

### 🔧 Integration with Your Codebase Agent:

Your project now includes:
- `memory_manager.py` - Core memory functionality
- `conversation_manager.py` - Enhanced with memory capabilities  
- Updated `main.py` - New memory-enabled API endpoints

### 🚀 New API Endpoints Available:
- `/chat-with-memory` - Memory-aware conversations
- `/analyze-repo-with-memory` - Repository analysis with user context
- `/suggest-feature-with-memory` - Feature suggestions based on preferences
- `/memory-search` - Search through user memories

### 💡 Benefits for Your Users:
- **Personalized Experience**: Recommendations based on past interactions
- **Context Continuity**: Maintains project context across sessions
- **Learning System**: Gets better at understanding user preferences over time
- **Repository-Specific Memory**: Different contexts for different projects

### 🔮 Next Steps:
1. Set your `GEMINI_API_KEY` environment variable
2. Install the updated requirements: `pip install -r requirements.txt`
3. Start your FastAPI server and test the new memory endpoints
4. Experiment with the interactive chat functionality

The memory system will make your codebase agent much more intelligent and user-friendly!